In [108]:
import sklearn
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

# Datasets import

In [118]:
aisles               =      pd.read_csv('~/kaggle-competitions/Instacart/data/aisles.csv')
departments          = pd.read_csv('~/kaggle-competitions/Instacart/data/departments.csv')
order_products_prior = pd.read_csv('~/kaggle-competitions/Instacart/data/order_products__prior.csv')
order_products_train = pd.read_csv('~/kaggle-competitions/Instacart/data/order_products__train.csv')
orders               = pd.read_csv('~/kaggle-competitions/Instacart/data/orders.csv')
products             = pd.read_csv('~/kaggle-competitions/Instacart/data/products.csv')

# Cleaning Datasets 

In [138]:
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


# Functions

In [3]:
def hours_to_moment(hour):
    try : 
        hour = int(hour)
        if hour>6 and hour<12:
            return 'morning'
        elif hour>=12 and hour<19:
            return 'afternoon'
        else : 
            return 'night'
    except : 
        return 'tbd'

In [54]:
orders['moments']= orders.order_hour_of_day.apply(hours_to_moment)

In [61]:
user_counter_dow_prior = orders[orders.eval_set=='prior'].groupby('user_id')['order_dow'].apply(list).apply(Counter).to_frame(name='dow')

In [62]:
user_nb_order_prior = orders[orders.eval_set=='prior'].groupby('user_id')['order_dow'].count().to_frame(name='nb_order')

In [63]:
user_counter_moment_prior = orders[orders.eval_set=='prior'].groupby('user_id')['moments'].apply(list).apply(Counter).to_frame(name='moments')

d = {'dow':counter_dow.values}
user_counter_dow = pd.DataFrame(data = d,index =counter_dow.index)

d = {'#order':nb_order.values}
user_nb_order = pd.DataFrame(data = d,index =nb_order.index)

d = {'moments':counter_moment.values}
user_counter_moment = pd.DataFrame(data = d,index =counter_moment.index)

users = user_counter_dow.join(user_nb_order, how='outer').join(user_counter_moment, how='outer')

In [64]:
users_prior = pd.concat([user_counter_dow_prior,user_nb_order_prior,user_counter_moment_prior],axis=1)

In [65]:
users_prior['nb_moments'] = users.moments.apply(len)

In [66]:
users_prior['nb_dow'] = users.dow.apply(len)

users[(users['#dow']==1) & (users['#moments']==1)]

In [81]:
users_prior.head()

,dow,nb_order,moments,nb_moments,nb_dow
user_id,,,,,
1,"{2: 2, 3: 2, 4: 3, 1: 3}",10,"{'morning': 6, 'afternoon': 4}",2,4
2,"{2: 5, 5: 1, 1: 5, 3: 2, 4: 1}",14,"{'morning': 12, 'afternoon': 2}",2,5
3,"{1: 2, 3: 3, 2: 1, 0: 6}",12,"{'afternoon': 11, 'night': 1}",2,4
4,"{6: 1, 4: 2, 5: 2}",5,"{'morning': 2, 'afternoon': 3}",2,3
5,"{3: 2, 0: 1, 1: 1}",4,{'afternoon': 4},1,3


In [131]:
order_prior_enriched = order_products_prior.join(products['department_id'],on='product_id', how='left')

In [137]:
order_prior_enriched['department_id'] = order_prior_enriched['department_id'].fillna(value=21).apply(int)

In [150]:
order_prior_enriched = order_prior_enriched.join(order_prior_enriched.groupby('order_id')['department_id'].nunique().to_frame(name='nb_unique_department'),how='outer',on='order_id')

In [157]:
orders = orders.join(order_prior_enriched['nb_unique_department'],on='order_id',how='left')

In [158]:
orders[orders.eval_set=='prior'].groupby('user_id')['nb_unique_department'].apply(list).apply(Counter).to_frame(name='number_department')

,number_department
user_id,
1,"{11: 2, 6: 3, 7: 2, 12: 1, 2: 1, 9: 1}"
2,"{12: 2, 7: 2, 11: 1, 13: 1, 6: 2, 4: 1, 9: 2, ..."
3,"{5: 1, 11: 2, 9: 3, 8: 4, 3: 1, 14: 1}"
4,"{1: 1, 10: 1, 7: 1, 9: 1, 3: 1}"
5,"{9: 1, 14: 1, 11: 1, 8: 1}"
6,"{13: 1, 9: 1, 11: 1}"
7,"{7: 6, 13: 1, 15: 1, 2: 3, 9: 3, 3: 1, 11: 1, ..."
8,"{7: 1, 2: 1, 9: 1}"
9,"{12: 1, 9: 1, 3: 1}"


In [159]:
orders[orders.eval_set=='prior'].groupby('user_id')['nb_unique_department'].max().

,max_number_department
user_id,
1,12
2,14
3,14
4,10
5,14
6,13
7,15
8,9
9,12


In [173]:
f = {'mean_nb_department' :np.mean,'max_nb_department' :np.max, 'min_nb_department':np.min, 'std_nb_departmenp.std()}
user_dep_prior = orders[orders.eval_set=='prior'].groupby('user_id')['nb_unique_department'].agg(f)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [174]:
users_prior = pd.concat([users_prior,user_dep_prior],axis=1)

In [177]:
users_prior.head()

,dow,nb_order,moments,nb_moments,nb_dow,mean_nb_department,max_nb_department,min_nb_department
user_id,,,,,,,,
1,"{2: 2, 3: 2, 4: 3, 1: 3}",10,"{'morning': 6, 'afternoon': 4}",2,4,7.700000,12,2
2,"{2: 5, 5: 1, 1: 5, 3: 2, 4: 1}",14,"{'morning': 12, 'afternoon': 2}",2,5,9.071429,14,3
3,"{1: 2, 3: 3, 2: 1, 0: 6}",12,"{'afternoon': 11, 'night': 1}",2,4,8.583333,14,3
4,"{6: 1, 4: 2, 5: 2}",5,"{'morning': 2, 'afternoon': 3}",2,3,6.000000,10,1
5,"{3: 2, 0: 1, 1: 1}",4,{'afternoon': 4},1,3,10.500000,14,8
